　「RとStanで始めるベイズ統計モデリングによるデータ分析入門」「実践編第4部第1章 階層ベイズモデルと一般化線形混合モデルの基本」を対象に，公開されているR，Stanのコードをpython，pystanのコードへと書き直した一例です。Stanの代わりにpystanを利用しています。

　この章では，階層ベイズモデルの基本的な事柄が解説されています。また階層ベイズモデルの具体例として，一般化線形混合モデル（Generalized Linear Mixed Models GLMM）の推定が試みられています。

　本ページでは公開されていない書籍の内容については一切触れません。理論や詳しい説明は書籍を参照してください。

　なお，こちらで紹介しているコードには誤りが含まれる可能性があります。内容やコードについてお気づきの点等ございましたら，ご指摘いただけると幸いです

# 分析の準備
## パッケージの読み込み

In [1]:
import arviz
import pystan
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'Meiryo'
import seaborn as sns

# データの読み込み
## 分析対象のデータ

In [49]:
fish_num_climate_2 = pd.read_csv('4-1-1-fish-num-2.csv')
fish_num_climate_2 = fish_num_climate_2.sort_values('temperature').reset_index(drop=True)

## id列を数値ではなくfactorとして扱う

In [50]:
fish_num_climate_2['id'] = fish_num_climate_2['id'].astype(object)
fish_num_climate_2.head(n=3)

,fish_num,weather,temperature,id
0,0,sunny,0.3,84
1,0,sunny,0.4,91
2,0,cloudy,0.5,22


In [51]:
fish_num_climate_2.dtypes

fish_num         int64
weather         object
temperature    float64
id              object
dtype: object

# 通常のポアソン回帰モデルの当てはめ

## 参考：デザイン行列の作成

In [52]:
# 応答変数の削除とダミー変数化処理
design_mat = pd.get_dummies(fish_num_climate_2.drop(['fish_num'], axis=1),
                            drop_first=True)

# 列名を書籍準拠に
design_mat = design_mat[['weather_sunny', 'temperature']]

# (Intercept)列追加
design_mat.insert(0, '(Intercept)', 1)

display(design_mat)

,(Intercept),weather_sunny,temperature
0,1,1,0.3
1,1,1,0.4
2,1,0,0.5
3,1,0,1.4
4,1,1,1.7
...,...,...,...
95,1,0,27.3
96,1,1,27.6
97,1,1,28.7
98,1,1,29.0


## 参考：データの作成

In [53]:
data_list = dict(N=len(fish_num_climate_2),
                 fish_num=fish_num_climate_2['fish_num'],
                 temp=fish_num_climate_2['temperature'],
                 sunny=design_mat['weather_sunny'])

## ポアソン回帰モデルを作る

In [64]:
# stanコードの記述
stan_code = '''
data {
  int N;                   // サンプルサイズ
  int fish_num[N];         // 釣獲尾数
  vector[N] temp;          // 気温データ
  vector[N] sunny;         // 晴れダミー変数
}

parameters {
  real Intercept;      // 切片
  real temperature;         // 係数(気温)
  real weathersunny;        // 係数(晴れの影響)
}

model {
  vector[N] lambda = exp(Intercept + temperature*temp + weathersunny*sunny);
  fish_num ~ poisson(lambda);
}

generated quantities {
    vector[N] lambda_sunny;
    vector[N] lambda_cloudy;
    vector[N] fish_num_sunny;
    vector[N] fish_num_cloudy;
    
    for(i in 1:N){
        lambda_sunny[i] = Intercept/10 + temperature*temp[i]/10 + weathersunny*1;
        lambda_cloudy[i] = Intercept/10 + temperature*temp[i]/10 + weathersunny*0;
        fish_num_sunny[i] = poisson_log_lpmf(lambda_sunny[i]);
        fish_num_cloudy[i] = poisson_log_lpmf(lambda_cloudy[i]);
    }
}

'''

# モデルのコンパイル
stan_model = pystan.StanModel(model_code=stan_code)

# サンプリング
glm_pois_brms = stan_model.sampling(data=data_list, seed=10, n_jobs=1)

ValueError: Failed to parse Stan model 'anon_model_6b4344472504b7fd6dfa1ce05b81426a'. Error message:
PARSER FAILED TO PARSE INPUT COMPLETELY
STOPPED AT LINE 20: 
transformed parameters {
    vector[N] lambda_sunny;
    vector[N] lambda_cloudy;
    for(i in 1:N){
        lambda_sunny[i] = Intercept/10 + temperature*temp[i]/10 + weathersunny*1;
        lambda_cloudy[i] = Intercept/10 + temperature*temp[i]/10 + weathersunny*0;
    }
}

generated quantities {
    vector[N] fish_num_sunny;
    vector[N] fish_num_cloudy;
    
    for(i in 1:N){
        fish_num_sunny[i] = poisson_log_rng(lambda_sunny[i]);
        fish_num_cloudy[i] = poisson_log_rng(lambda_cloudy[i]);
    }
}







## 参考：結果の表示

In [16]:
print(glm_pois_brms.stansummary(probs=[0.025, 0.5, 0.975]))

Inference for Stan model: anon_model_10b93b39bc827a53de74eb5431c131b8.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

            mean se_mean     sd    2.5%    50%  97.5%  n_eff   Rhat
Intercept    0.3  3.7e-3   0.16 -8.9e-3    0.3    0.6   1775    1.0
b_temp      0.06  1.8e-4 7.9e-3    0.05   0.06   0.08   1885    1.0
b_sunny    -0.74  2.7e-3   0.13    -1.0  -0.74  -0.49   2377    1.0
lp__       33.94    0.03   1.28   30.57  34.28  35.35   1405    1.0

Samples were drawn using NUTS at Sun Sep 13 21:04:47 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


## 結果の図示

In [18]:
mcmc_sample = glm_pois_brms.extract()

In [20]:
mcmc_sample

OrderedDict([('Intercept',
              array([0.64283879, 0.50486541, 0.11216955, ..., 0.27917243, 0.3700465 ,
                     0.2987611 ])),
             ('b_temp',
              array([0.0396495 , 0.05343667, 0.06961966, ..., 0.05974184, 0.04966696,
                     0.05223113])),
             ('b_sunny',
              array([-0.80894853, -0.86805635, -0.72182174, ..., -0.70892262,
                     -0.56269859, -0.50628397])),
             ('lp__',
              array([30.97087171, 34.15544169, 34.68382545, ..., 35.35062453,
                     33.51720881, 33.19241604]))])

In [35]:
help(pystan.StanModel.sampling)

Help on function sampling in module pystan.model:

sampling(self, data=None, pars=None, chains=4, iter=2000, warmup=None, thin=1, seed=None, init='random', sample_file=None, diagnostic_file=None, verbose=False, algorithm=None, control=None, n_jobs=-1, **kwargs)
    Draw samples from the model.
    
    Parameters
    ----------
    data : dict
        A Python dictionary providing the data for the model. Variables
        for Stan are stored in the dictionary as expected. Variable
        names are the keys and the values are their associated values.
        Stan only accepts certain kinds of values; see Notes.
    
    pars : list of string, optional
        A list of strings indicating parameters of interest. By default
        all parameters specified in the model will be stored.
    
    chains : int, optional
        Positive integer specifying number of chains. 4 by default.
    
    iter : int, 2000 by default
        Positive integer specifying how many iterations for each chai

In [61]:
np.exp(20.8)

1079754999.464535